In [21]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import random
import csv, os

pygame.init()

(5, 0)

In [22]:
ISLAND_MAX_SIZE = 150
ISLAND_MIN_SIZE = 50
ISLAND_COUNT = 3
SPECIES_COUNT = 100
SPECIES_POPULATION = 1
INDIVIDUAL_R = 5
MOVEMENT_SPEED = 3
TOTAL_POP_SIZE = SPECIES_COUNT * SPECIES_POPULATION
REPLICATION_TIME = 50

In [23]:
MAINLAND_HEIGHT = 50
OCEAN_HEIGHT = 800
WIDTH, HEIGHT = 800, MAINLAND_HEIGHT + OCEAN_HEIGHT
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
BLACK = (0, 0, 0)
BLUE1 = (11, 154, 189)
BLUE2 = (49, 197, 197)
BLUE3 = (143, 223, 211)
BROWN1 = (83, 59, 51)
BROWN2 = (131, 105, 83)
YELLOW = (255, 255, 170)
RED = (255, 0, 0)
WIN.fill(BLUE1)
pygame.draw.rect(WIN, BROWN1, (0, OCEAN_HEIGHT , 800, MAINLAND_HEIGHT))
pygame.display.update()


In [24]:
class Island:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = random.randint(ISLAND_MIN_SIZE, ISLAND_MAX_SIZE)
        self.color = BROWN2
        self.population = 0
        self.area = math.pi * self.r ** 2
        self.capacity = int(self.area / 10)
        self.avaiable = True
    
    def draw(self):
        pygame.draw.circle(WIN, self.color, (self.x, self.y), self.r)
        pygame.draw.circle(WIN, YELLOW, (self.x, self.y), self.r + 1, width=3)
        pygame.draw.circle(WIN, BLUE3, (self.x, self.y), self.r + 4, width=3)
        pygame.draw.circle(WIN, BLUE2, (self.x, self.y), self.r + 7, width=3)
        pygame.display.update(self.x - INDIVIDUAL_R, self.y - INDIVIDUAL_R, 10, 10)


    def generate(count, islands):
        while len(islands) < count:
            x = random.randint(ISLAND_MAX_SIZE, WIDTH - ISLAND_MAX_SIZE)
            y = random.randint(ISLAND_MAX_SIZE, OCEAN_HEIGHT - ISLAND_MAX_SIZE)
            new_island = Island(x, y)
            if all(np.sqrt((new_island.x - island.x)**2 + (new_island.y - island.y)**2) >= (new_island.r + island.r) for island in islands):
                islands.append(new_island)        
        return islands
    
    def check_avaiable(self):
        if self.population < self.capacity:
            self.avaiable = True
        else:
            self.avaiable = False

    def extinction(self, individuals):
        if not self.avaiable:
            difference = int(self.population - self.capacity)
            ind_on_island = [individual for individual in individuals if individual.island == self]
            ind_to_remove = random.sample(ind_on_island, difference)
            for ind in ind_to_remove:
                individuals.remove(ind)
                self.population -= 1
        self.check_avaiable()
        return individuals

In [25]:
class Species:
    def __init__(self):
        self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        self.population = SPECIES_POPULATION
        self.pop_size = 0

    def generate(count,species):
        while len(species) < count:
            new_species = Species()
            species.append(new_species)
        return species

In [26]:
class Individuals:
    def __init__(self, species):
        self.x = None
        self.y = None
        self.species = species
        self.color = self.species.color
        self.island = None
    
    def generate(count, species, individuals):
        for specie in species:
            for i in range(count):
                new_individual = Individuals(specie)
                new_individual.x = random.randint(0, WIDTH)
                new_individual.y = random.randint(OCEAN_HEIGHT, OCEAN_HEIGHT + MAINLAND_HEIGHT)
                specie.pop_size += 1
                individuals.append(new_individual)
        return individuals
    
    def draw(self):
        if not self.island:
            pygame.draw.circle(WIN, self.color, (self.x, self.y), INDIVIDUAL_R)
        else:
            pygame.draw.circle(WIN, self.color, (self.x, self.y), INDIVIDUAL_R)

    def draw_island(self):
        while True:
            self.x = np.random.randint(self.island.x - self.island.r, self.island.x + self.island.r)
            self.y = np.random.randint(self.island.y - self.island.r, self.island.y + self.island.r)
            if np.sqrt((self.x - self.island.x)**2 + (self.y - self.island.y)**2) <= self.island.r:
                pygame.draw.circle(WIN, self.color, (self.x, self.y), INDIVIDUAL_R)
                break

    def move(self, islands):
        if not self.island:
            old_x = self.x
            old_y = self.y
            pygame.draw.circle(WIN, BROWN1, (self.x, self.y), INDIVIDUAL_R)
            pygame.display.update(self.x - INDIVIDUAL_R, self.y - INDIVIDUAL_R, 10, 10)
            self.x += np.random.randint(-MOVEMENT_SPEED, MOVEMENT_SPEED + 1)
            self.y += np.random.randint(-MOVEMENT_SPEED, MOVEMENT_SPEED + 1)
            self.x = min(self.x, WIDTH)
            self.y = min(self.y, HEIGHT)
            if OCEAN_HEIGHT > self.y:
                distances_list = []
                avaiable_islands = [island for island in islands if island.avaiable]
                for island in avaiable_islands:
                    distance = np.sqrt((self.x - island.x)**2 + (self.y - island.y)**2)
                    distances_list.append(1 / distance)
                if distances_list:
                    sum_distances = sum(distances_list)
                    probabilities = [distance / sum_distances for distance in distances_list]
                    target_island = np.random.choice(avaiable_islands, p=probabilities)
                else:
                    target_island = None
                    self.x = old_x
                    self.y = old_y
                if target_island:
                    self.island = target_island
                    Individuals.draw_island(self)
                    self.island.population += 1

    def reproduction(individuals):
        new_individuals = []
        random.shuffle(individuals)
        for ind in individuals:
            if ind.island:
                new_individual = Individuals(ind.species)
                new_individual.island = ind.island
                new_individual.draw_island()
                new_individuals.append(new_individual)
                ind.island.population += 1
        individuals.extend(new_individuals)
        return individuals


In [27]:
def main():
    csv_file_path = "values.csv"
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
    
    run = True
    clock = pygame.time.Clock()
    islands = []
    paused = True
    time = 0

    species = Species.generate(SPECIES_COUNT, [])
    individuals = Individuals.generate(SPECIES_POPULATION, species, [])
    islands_distance = []
    islands_size = []
    island_capacity = []
    
    with open(csv_file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([
            "time",
            "islands_size",
            "island_capacity",
            "islands_distance",
            "islands_diversity",
            "individual_count"
            ])   

        while run:
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                if event.type == pygame.MOUSEBUTTONDOWN:
                    x, y = pygame.mouse.get_pos()
                    new_island = Island(x, y)
                    islands.append(new_island)
                    islands_size.append(new_island.area)
                    islands_distance.append(np.sqrt((new_island.x - WIDTH / 2)**2 + (new_island.y - OCEAN_HEIGHT)**2))
                    island_capacity.append(new_island.capacity)
                    new_island.draw()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        paused = not paused
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_c:
                        paused = True
                        islands = []
                        WIN.fill(color= BLUE1 , rect = (0, 0, WIDTH, OCEAN_HEIGHT))
                        species = Species.generate(SPECIES_COUNT, [])
                        individuals = Individuals.generate(SPECIES_POPULATION, species, [])
                        for ind in individuals:
                            ind.draw()
            if not paused:
                time += 1
                if time % 1 == 0:
                    island_species_count = []
                    island_individuals_count = []
                    for island in islands:
                        island_species = []
                        island_individuals = 0
                        for ind in individuals:
                            if ind.island == island:
                                island_individuals += 1
                                if ind.species not in island_species:
                                    island_species.append(ind.species)
                        island_species_count.append(len(island_species))
                        island_individuals_count.append(island_individuals)

                    writer.writerow([time,
                                    islands_size,
                                    island_capacity,
                                    islands_distance,
                                    island_species_count,
                                    island_individuals_count])

                WIN.fill(color= BLUE1 , rect = (0, 0, WIDTH, OCEAN_HEIGHT))
                for ind in individuals:
                    ind.move(islands)
                for island in islands:
                    island.check_avaiable()
                    island.extinction(individuals)
                    island.draw()
                if time % REPLICATION_TIME == np.inf:
                    individuals = Individuals.reproduction(individuals)
                for ind in individuals:
                    ind.draw()
            pygame.display.set_caption("Island Biogeography | " + 
                                    "Islands: " + str(len(islands)) + 
                                    " Species: " + str(len(species)) + 
                                    " Individuals: " + str(len(individuals)))
            
        
            pygame.display.update()
            clock.tick(60)
        print(len(individuals))
        pygame.quit()

if __name__ == "__main__":
    main()
            

100


In [28]:
df = pd.read_csv("values.csv")
df1 = df.copy()
df["species_change"] = pd.NA

In [29]:
def StringtoList(string):
    x = string.strip('][').split(', ')
    for i in range(len(x)):
        x[i] = float(x[i])
    return x

In [30]:
for row in range(len(df)):
    df.islands_size[row] = StringtoList(df.islands_size[row])
    df.island_capacity[row] = StringtoList(df.island_capacity[row])
    df.islands_distance[row] = StringtoList(df.islands_distance[row])
    df.islands_diversity[row] = StringtoList(df.islands_diversity[row])
    df.individual_count[row] = StringtoList(df.individual_count[row])
    if row == 0:
        df.species_change[row] = [0,0,0]
    else:
        change = [df.islands_diversity[row][i] - df.islands_diversity[row - 1][i] for i in range(len(df.islands_diversity[row]))]
        df.species_change[row] = change


C:\Users\kayra\AppData\Local\Temp\ipykernel_2948\900083496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.islands_size[row] = StringtoList(df.islands_size[row])
C:\Users\kayra\AppData\Local\Temp\ipykernel_2948\900083496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.island_capacity[row] = StringtoList(df.island_capacity[row])
C:\Users\kayra\AppData\Local\Temp\ipykernel_2948\900083496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

IndexError: list index out of range

-[] 